# Preprocesamiento del corpus


Leemos el documento y lo cargamos en un dataframe

In [69]:
import csv
import pandas as pd


# Lee el archivo CSV y crea el DataFrame
movies_df = pd.read_csv('/rotten_tomatoes_movies.csv')
reviews_df = pd.read_csv('/rotten_tomatoes_critic_reviews.csv')

# Muestra el DataFrame
# movies_df.head()
# reviews_df.head()

Reviso si me muestra la review

In [70]:
x = reviews_df.iloc[0]['review_content']
x

'A fantasy adventure that fuses Greek mythology to contemporary American places and values. Anyone around 15 (give or take a couple of years) will thrill to the visual spectacle'

Uso una librería para las stopwords y una función para limpiar el texto

In [71]:
import re
from nltk.corpus import stopwords

# Descargar las stopwords
import nltk
nltk.download('stopwords')

# Normalización del texto con manejo de valores no-string o nulos
def clean_text(text):
    if isinstance(text, str):  # Verificar si es una cadena
        text = text.lower()  # Convertir a minúsculas
        text = re.sub(r'[^\w\s]', '', text)  # Eliminar puntuaciones
        text = re.sub(r'\s+', ' ', text).strip()  # Eliminar espacios extras
        stop_words = set(stopwords.words('english'))
        text = ' '.join([word for word in text.split() if word not in stop_words])  # Quitar stopwords
        return text
    else:
        return ''  # Retornar vacío si no es texto

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Aplico la función para limpiar el texto

In [72]:
reviews_df['cleaned_review'] = reviews_df['review_content'].apply(clean_text)

Así se ve limpio

In [73]:
x = reviews_df.iloc[0]['cleaned_review']
print(x)

fantasy adventure fuses greek mythology contemporary american places values anyone around 15 give take couple years thrill visual spectacle


In [74]:
# Unir reseñas con información de películas usando rotten_tomatoes_link
reviews_with_movies = reviews_df.merge(movies_df, on='rotten_tomatoes_link', how='inner')

# Crear un campo combinado si es necesario (solo para futuras optimizaciones)
reviews_with_movies['combined_text'] = reviews_with_movies['cleaned_review']


In [75]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Vectorizar las reseñas (TF-IDF)
vectorizer_reviews = TfidfVectorizer()

# Crear la matriz TF-IDF basada en las reviews preprocesadas
tfidf_matrix_reviews = vectorizer_reviews.fit_transform(reviews_with_movies['combined_text'])

In [79]:
def search_movies_by_reviews(query, vectorizer, tfidf_matrix, reviews_df, top_n=5):
    # Normalizar y vectorizar la consulta
    query_cleaned = clean_text(query)
    query_vec = vectorizer.transform([query_cleaned])

    # Calcular la similitud coseno
    similarity_scores = cosine_similarity(query_vec, tfidf_matrix).flatten()

    # Obtener índices de las reseñas más relevantes
    top_indices = similarity_scores.argsort()[-top_n:][::-1]

    # Recuperar los IDs de las películas relevantes
    relevant_movies = reviews_df.iloc[top_indices]['rotten_tomatoes_link'].values

    # Filtrar títulos de películas y garantizar que coincidan con los IDs relevantes
    results = movies_df[movies_df['rotten_tomatoes_link'].isin(relevant_movies)][['movie_title', 'genres']]

    # Asignar similitudes correspondientes solo a los IDs encontrados
    results = results.reset_index(drop=True)
    results['Similarity'] = similarity_scores[top_indices[:len(results)]]

    return results


In [80]:
# Probar con una consulta
query = "Amazing story and breathtaking visuals"
results = search_movies_by_reviews(query, vectorizer_reviews, tfidf_matrix_reviews, reviews_with_movies)

# Mostrar resultados
print("Resultados para la consulta:", query)
print(results)


Resultados para la consulta: Amazing story and breathtaking visuals
   movie_title                            genres  Similarity
0   Crazy Love                       Documentary    0.611086
1         Race                             Drama    0.611086
2  The Soloist  Drama, Musical & Performing Arts    0.611086


# Posibles mejoras

1.   Agregar resultados de críticas

Es buena idea agregar resultados de críticas de críticos. Esta información es de relevancia para el usuario, pues le permite tener un mejor criterio a la hora de elegir una película

2.   Interfaz interactiva

Siempre es útil la inclusión de una interfaz para ser más amigable con el usuario







